In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON, SPARQLExceptions
import requests
import re
from rdflib import Graph
import rdflib
# from rdfpandas.graph import to_dataframe
import pandas as pd
from tqdm.auto import tqdm
from datetime import datetime
from rdflib import Graph

from multiprocessing import Pool, cpu_count
import json
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from tqdm.auto import tqdm
tqdm.pandas()
import os
from collections import defaultdict
import pyarrow as pa
import pyarrow.parquet as pq
import numpy as np
from collections import Counter
import ast

In [2]:
def parse_string_list(s):
    if isinstance(s, str):
        if s.startswith('[') and s.endswith(']'):
            try:
                return ast.literal_eval(s)
            except:
                pass
    return s

def process_json_object(json_object):
    if '@graph' in json_object:
        df = pd.json_normalize(json_object['@graph'])
        for col in df.columns:
            if df[col].dtype == 'object':
                df[col] = df[col].apply(parse_string_list)
        return df
    return pd.DataFrame()

def process_file(filename):
    with open(filename, 'r', encoding='utf-8') as file:
        content = file.read()
    
    # 여러 JSON 객체를 분리
    json_objects = content.split('}\n{')
    
    # 첫 번째와 마지막 객체의 중괄호 복원
    json_objects[0] = json_objects[0] + '}'
    json_objects[-1] = '{' + json_objects[-1]
    
    # 중간 객체들의 중괄호 복원
    for i in range(1, len(json_objects) - 1):
        json_objects[i] = '{' + json_objects[i] + '}'
    
    dataframes = []
    for json_str in tqdm(json_objects, desc="JSON 객체 처리"):
        try:
            json_obj = json.loads(json_str)
            df = process_json_object(json_obj)
            if not df.empty:
                dataframes.append(df)
        except json.JSONDecodeError as e:
            print(f"JSON 디코딩 오류: {e}")
    
    if dataframes:
        return pd.concat(dataframes, ignore_index=True)
    else:
        return pd.DataFrame()

# 설정 변수
# filename = 'PersonMaterial_json_20240413.json'  # 처리할 파일 이름
filename = 'Organization_0.json'  # 처리할 파일 이름

# 파일 처리
df = process_file(filename)

print("처리 완료!")
print(f"DataFrame 크기: {df.shape}")
display(df.head())

JSON 객체 처리:   0%|          | 0/15 [00:00<?, ?it/s]

처리 완료!
DataFrame 크기: (144800, 42)


,@id,rdf:type,homepage,mbox,member,name,@type,dateOfClosed,dateOfOpening,isClosed,...,url,sameAs,altLabel,prefLabel,corporateName,relatedOrganization,laterCorportate,personalName,relatedPerson,birthPlace
0,http://listinc.kr/id/listinc,"[http://www.w3.org/ns/org#Organization, http:/...","[http://li-st.com/, http://listinc.kr/]","[mailto:hello@listinc.kr, mailto:hello@li-st.com]","[http://listinc.kr/id/member/joyhong, http://l...","[주식회사 리스트@ko, LiST Inc.@en]",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,nllib:011001,"[http://www.w3.org/ns/org#Organization, http:/...",http://www.nl.go.kr,NaN,NaN,"[국립중앙도서관@ko, National Library of Korea@en]",nlon:Library,"매월 둘째 네째 월요일, 공휴일",1945-10-15,false,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,nllib:011003,http://xmlns.com/foaf/0.1/Organization,http://www.nlcy.go.kr/,NaN,NaN,NaN,nlon:Library,NaN,NaN,false,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,nllib:011005,http://xmlns.com/foaf/0.1/Organization,NaN,NaN,NaN,NaN,nlon:Library,NaN,NaN,false,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,nllib:011010,http://xmlns.com/foaf/0.1/Organization,http://https://www.nld.go.kr,NaN,NaN,NaN,nlon:Library,NaN,2012/08/18,false,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# df.to_pickle('PersonMaterial_json_20240413.pkl')
# df.to_pickle('Organization_0.pkl')

In [14]:
df = pd.read_pickle('./PersonMaterial_json_20240413.pkl')
df_org = pd.read_pickle('./Organization_0.pkl')

In [19]:
df['birthYear'] = df['birthYear'].astype(str)  # Convert the 'birth' column to string type
df['birthYear'] = df['birthYear'].apply(lambda x: re.findall(r'\d+', x)[0] if re.findall(r'\d+', x) else None)
df['birthYear']

0          None
1          None
2          1952
3          1938
4          1898
           ... 
1635338    None
1635339    None
1635340    None
1635341    1971
1635342    1982
Name: birthYear, Length: 1635343, dtype: object

In [ ]:
# def add_name_variations(alt_labels):
#     new_variations = []
    
#     for label in alt_labels:
#         if ',' in label:
#             # 쉼표가 있는 경우 (예: "푸코, 미셸")
#             last_name, first_name = label.split(',')
#             new_variation = f"{first_name.strip()} {last_name.strip()}"
#             new_variations.append(new_variation)
#         elif ' ' in label and not label.endswith('.'):
#             # 공백이 있고 약어가 아닌 경우 (예: "Paul Michel Foucault")
#             names = label.split()
#             if len(names) > 1:
#                 new_variation = f"{names[-1]}, {' '.join(names[:-1])}"
#                 new_variations.append(new_variation)
    
#     # 중복 제거 및 기존 리스트와 새로운 변형 합치기
#     return list(set(alt_labels + new_variations))

# # 테스트
# alt_labels = ["푸코, 미셸", "Foucault, Paul Michel", "Fuḳo, Mishel", "フーコー, ミシェル", "푸꼬, 미셸", "Foucault, M."]
# result = add_name_variations(alt_labels)

# print("원본 altLabel:")
# print(alt_labels)
# print("\n확장된 altLabel:")
# print(result)

In [6]:
def add_name_variations(pref_name, alt_labels):
    def process_name(name):
        # @과 그 뒤의 언어 태그 제거
        name = re.sub(r'@\w+$', '', name)
        
        variations = [name]
        if ',' in name:
            parts = name.split(',')
            last_name = parts[0].strip()
            first_name = ' '.join(part.strip() for part in parts[1:])
            variations.append(f"{first_name} {last_name}")
        elif ' ' in name and not name.endswith('.'):
            names = name.split()
            if len(names) > 1:
                variations.append(f"{names[-1]}, {' '.join(names[:-1])}")
        return variations

    new_variations = []
    
    if isinstance(pref_name, str):
        new_variations.extend(process_name(pref_name))
    
    if alt_labels is None:
        alt_labels = []
    elif isinstance(alt_labels, str):
        alt_labels = [alt_labels]
    elif not isinstance(alt_labels, list):
        alt_labels = []

    for label in alt_labels:
        if isinstance(label, str):
            new_variations.extend(process_name(label))

    # 중복 제거 및 정렬
    return sorted(set(new_variations))

# 테스트
result = add_name_variations('Foucault, Michel', ["푸코, 미셸", "Foucault, Paul Michel", "Fuḳo, Mishel", "フーコー, ミシェル", "푸꼬, 미셸", "Foucault, M.", "權寧珉@zh"])
print(result)

['Foucault, M.', 'Foucault, Michel', 'Foucault, Paul Michel', 'Fuḳo, Mishel', 'M. Foucault', 'Michel Foucault', 'Mishel Fuḳo', 'Paul Michel Foucault', 'フーコー, ミシェル', 'ミシェル フーコー', '權寧珉', '미셸 푸꼬', '미셸 푸코', '푸꼬, 미셸', '푸코, 미셸']


In [5]:
df['newAltlabel'] = df.progress_apply(lambda row: add_name_variations(row['prefLabel'], row['altLabel']), axis=1)

  0%|          | 0/1635343 [00:00<?, ?it/s]

In [6]:
# ref_df = pd.read_csv('./final_output.csv')
ref_df = pd.read_csv('./updated_final_output.csv')
ref_df.head()

,refebibl-id,type-code,type-name,title,author,journal-name,pubi-year,volume,isseue,serno,...,department,artiId,url,citation-day,site-name,registration-day,conference-name,modern_liter,unified_title,title_index
0,REF041663861,3,단행본,역사 속의 페미니스트,거다 러너,NaN,1993,NaN,NaN,NaN,...,NaN,ART002094298,NaN,NaN,NaN,NaN,NaN,추가,역사 속의 페미니스트,0
1,REF041663862,3,단행본,가사집2,고정옥,NaN,1996,NaN,NaN,NaN,...,NaN,ART002094298,NaN,NaN,NaN,NaN,NaN,추가,가사집2,1
2,REF041663863,3,단행본,가사문학대계,권영철,NaN,1979,NaN,NaN,NaN,...,NaN,ART002094298,NaN,NaN,NaN,NaN,NaN,추가,가사문학대계,2
3,REF041663864,3,단행본,규방가사연구,권영철,NaN,1980,NaN,NaN,NaN,...,NaN,ART002094298,NaN,NaN,NaN,NaN,NaN,추가,규방가사연구,3
4,REF041663865,3,단행본,注解歌辭文學全集,金聖配,NaN,1979,NaN,NaN,NaN,...,NaN,ART002094298,NaN,NaN,NaN,NaN,NaN,추가,注解歌辭文學全集,4


In [11]:
ref_df.columns

Index(['refebibl-id', 'type-code', 'type-name', 'title', 'author',
       'journal-name', 'pubi-year', 'volume', 'isseue', 'serno', 'page',
       'pubilisher', 'doi', 'arti-id', 'degree', 'university', 'department',
       'artiId', 'url', 'citation-day', 'site-name', 'registration-day',
       'conference-name', 'modern_liter', 'unified_title', 'title_index'],
      dtype='object')

In [14]:
ref_df = ref_df.drop_duplicates(subset='refebibl-id').reset_index(drop=True)

In [15]:
ref_df[ref_df.duplicated('refebibl-id')]

,refebibl-id,type-code,type-name,title,author,journal-name,pubi-year,volume,isseue,serno,...,department,artiId,url,citation-day,site-name,registration-day,conference-name,modern_liter,unified_title,title_index


In [17]:
# 현대문학만 필터링
modern_df = pd.read_pickle('./240731_2008_2022_한국현대문학_통합_성별_생년.pkl')
modern_df

,article-id,gender,birth
0,ART002536712,남,69
1,ART002536697,남,71
2,ART002536682,여,78
3,ART002536671,남,74
4,ART002536636,남,90
...,...,...,...
15922,ART002796203,NaN,NaN
15923,ART001536392,여,70
15924,ART001536458,여,65
15925,ART001941069,남,50


In [22]:
ref_df['artiId'].nunique()

411

In [27]:
# 현대문학 논문의 참고문헌만 필터링
ref_df = ref_df[ref_df['artiId'].isin(modern_df['article-id'].tolist())].reset_index(drop=True)

In [28]:
ref_df['artiId'].nunique()

368

In [42]:
len(ref_df)

11092

In [29]:
# 저자 문자열을 리스트로 변환하는 함수
def split_authors(authors_str):
    return [author.strip() for author in authors_str.split(';') if author.strip()]

# ref_df의 'author' 컬럼을 리스트로 변환
ref_df['author_list'] = ref_df['author'].dropna().apply(split_authors)

In [30]:
# 저자 매칭을 위한 딕셔너리 생성
author_dict = defaultdict(list)
for _, row in df.iterrows():
    if isinstance(row['newAltlabel'], list):
        for name in row['newAltlabel']:
            if isinstance(name, str):
                author_dict[name].append((row['prefLabel'], row['@id'], row.get('jobTitle', '')))

# 우선순위 직함 리스트
priority_jobs = ['교수(대학)[敎授]', '소설가[小說家]', '강사(선생)[講師]', '연구원(연구자)[硏究員]']

# 매칭 함수 정의
def match_author(author):
    if pd.isna(author) or not isinstance(author, str):
        return author, None
    
    if author in author_dict:
        matches = author_dict[author]
        
        # 우선순위 직함을 가진 매치 찾기
        priority_matches = [match for match in matches if any(job in str(match[2]) for job in priority_jobs)]
        
        if priority_matches:
            return priority_matches[0][:2]  # prefLabel과 @id만 반환
        elif matches:
            return matches[0][:2]  # prefLabel과 @id만 반환
    
    return author, None

# 저자 리스트를 처리하는 함수
def process_author_list(authors):
    if isinstance(authors, list):
        return [match_author(author) for author in authors if isinstance(author, str)]
    elif isinstance(authors, str):
        return [match_author(authors)]
    else:
        return []

# 진행 상황을 보여주며 ref_df의 각 저자 리스트에 대해 매칭 수행
tqdm.pandas(desc="저자 매칭 중")
ref_df['matched_results'] = ref_df['author_list'].progress_apply(process_author_list)

# 결과 분리
ref_df['matched_authors'] = ref_df['matched_results'].apply(lambda x: [item[0] for item in x])
ref_df['matched_ids'] = ref_df['matched_results'].apply(lambda x: [item[1] for item in x])

저자 매칭 중:   0%|          | 0/11092 [00:00<?, ?it/s]

In [31]:
# 수동으로 변경하고 싶은 저자와 새 ID를 포함하는 딕셔너리
manual_corrections = {
    "이선미": "nlk:KAC201228519",
    "김은하": "nlk:KAC201309002",
    "김은정":"nlk:KAC201429628",
    "신수정":"nlk:KAC201753589",
    "김미영":"nlk:KAC201809337", #중복 가능성 많음
    "김윤정":"nlk:KAC201310839",
    "김미현":"nlk:KAC201316649",
    "정미숙":"nlk:KAC201405879",
    "권영민":"nlk:KAC201214923",
    "신샛별":None,
    "이선옥":"nlk:KAC201426716",
    "천정환":"nlk:KAC201232410",
    "조한혜정":"nlk:KAC201429164",
    "조혜정":"nlk:KAC201429164",
    '이수형':'nlk:KAC201213091',
    '이상경':'nlk:KAC201320882',
    '한수영':'nlk:KAC201218014',
    '김현주':'nlk:KAC201762172'
}

# ref_df의 매칭 결과 업데이트 함수
def update_matches(row):
    updated_authors = []
    updated_ids = []
    for author, id in zip(row['matched_authors'], row['matched_ids']):
        if author in manual_corrections:
            if author == "조혜정":
                updated_authors.append("조한혜정")
            else:
                updated_authors.append(author)
            updated_ids.append(manual_corrections[author])
        else:
            updated_authors.append(author)
            updated_ids.append(id)
    return pd.Series({'matched_authors': updated_authors, 'matched_ids': updated_ids})

# 매칭 결과 업데이트
ref_df[['matched_authors', 'matched_ids']] = ref_df.apply(update_matches, axis=1)

In [32]:
# 결과 확인
print("업데이트된 매칭 결과 (처음 5개 행):")
print(ref_df[['matched_authors', 'matched_ids']].head())

# 수동으로 변경된 저자들의 결과 확인
manual_authors = set(manual_corrections.keys())
manual_results = ref_df[ref_df['matched_authors'].apply(lambda x: any(author in manual_authors for author in x))]
print("\n수동으로 변경된 저자들의 결과:")
print(manual_results[['matched_authors', 'matched_ids']])

# "조한혜정" 확인
cho_results = ref_df[ref_df['matched_authors'].apply(lambda x: "조한혜정" in x)]
print("\n'조한혜정' 결과:")
print(cho_results[['matched_authors', 'matched_ids']])

업데이트된 매칭 결과 (처음 5개 행):
  matched_authors         matched_ids
0           [염상섭]  [nlk:KAC201010743]
1           [염상섭]  [nlk:KAC201010743]
2           [권영민]  [nlk:KAC201214923]
3           [김경수]  [nlk:KAC201200936]
4           [김복순]  [nlk:KAC201212723]

수동으로 변경된 저자들의 결과:
      matched_authors         matched_ids
2               [권영민]  [nlk:KAC201214923]
6               [김윤정]  [nlk:KAC201310839]
47              [김윤정]  [nlk:KAC201310839]
94              [권영민]  [nlk:KAC201214923]
174             [김미현]  [nlk:KAC201316649]
...               ...                 ...
10965           [신샛별]              [None]
10966           [신수정]  [nlk:KAC201753589]
10971           [이수형]  [nlk:KAC201213091]
11017           [김은정]  [nlk:KAC201429628]
11086           [이선미]  [nlk:KAC201228519]

[625 rows x 2 columns]

'조한혜정' 결과:
      matched_authors         matched_ids
201            [조한혜정]  [nlk:KAC201429164]
455            [조한혜정]  [nlk:KAC201429164]
456            [조한혜정]  [nlk:KAC201429164]
457            [조한혜정] 

In [33]:
# 모든 매칭된 저자를 하나의 리스트로 합치기
all_matched_authors = [author for authors in ref_df['matched_authors'] for author in authors]
all_matched_ids = [id for ids in ref_df['matched_ids'] for id in ids]

# 저자와 ID를 짝지어 딕셔너리 생성 (ID가 있는 경우만)
author_id_dict = {author: id for author, id in zip(all_matched_authors, all_matched_ids) if not pd.isna(id)}

# 저자 빈도 계산
author_freq = Counter(all_matched_authors)

# 상위 100명의 저자 선택
top_100_authors = author_freq.most_common(100)

# 결과를 저장할 데이터프레임 생성
top_authors_df = pd.DataFrame(top_100_authors, columns=['Author', 'Frequency'])
top_authors_df['ID'] = top_authors_df['Author'].map(author_id_dict)

# 원본 데이터에서 추가 정보 가져오기
def get_author_info(row):
    if pd.isna(row['ID']):
        return pd.Series({'JobTitle': '', 'BirthYear': '', 'FieldOfActivity': '', 'Gender': ''})
    author_row = df[df['@id'] == row['ID']].iloc[0] if not df[df['@id'] == row['ID']].empty else pd.Series()
    return pd.Series({
        'JobTitle': author_row.get('jobTitle', ''),
        'BirthYear': author_row.get('birthYear', ''),
        'FieldOfActivity': author_row.get('fieldOfActivity', ''),
        'Gender': author_row.get('gender', '')  # 'gender' 컬럼에서 정보 가져오기
    })

author_info = top_authors_df.apply(get_author_info, axis=1)
top_authors_df = pd.concat([top_authors_df, author_info], axis=1)

# 결과 출력
print("상위 100명의 매칭된 저자:")
print(top_authors_df)

# 매칭 통계
total_authors = sum(len(authors) for authors in ref_df.dropna(subset=['author'])['author_list'])
matched_authors = sum(sum(1 for id in ids if not pd.isna(id)) for ids in ref_df['matched_ids'])
print(f"\n총 저자 수: {total_authors}")
print(f"매칭된 저자 수: {matched_authors}")
print(f"매칭 비율: {matched_authors/total_authors:.2%}")

# ID가 없는 저자 확인
authors_without_id = top_authors_df[top_authors_df['ID'].isna()]
print("\nID가 없는 저자 (상위 100명 중):")
print(authors_without_id)

# 매칭되지 않은 저자 확인
unmatched_authors = [author for authors in ref_df.dropna(subset=['author'])['author_list'] for author in authors if author not in author_dict]
print(f"\n매칭되지 않은 고유한 저자 수: {len(set(unmatched_authors))}")
print("매칭되지 않은 저자 예시 (최대 10개):")
print(list(set(unmatched_authors))[:10])

# 성별 통계 (ID가 있는 저자들 중에서)
gender_stats = top_authors_df[top_authors_df['ID'].notna()]['Gender'].value_counts()
print("\n성별 통계 (ID가 있는 저자들 중):")
print(gender_stats)
print(f"성별 비율: {gender_stats / gender_stats.sum() * 100}")

상위 100명의 매칭된 저자:
              Author  Frequency                ID  \
0                박완서        828  nlk:KAC201007995   
1                김윤식        110  nlk:KAC201112267   
2                이선미        109  nlk:KAC201228519   
3                김은하         89  nlk:KAC201309002   
4                김양선         77  nlk:KAC201214437   
..               ...        ...               ...   
95               변학수         16  nlk:KAC201229359   
96  Bourdieu, Pierre         15  nlk:KAC200001177   
97              권보드래         15  nlk:KAC201231579   
98               오혜진         15  nlk:KAC201217048   
99               이명호         15  nlk:KAC201210739   

                                             JobTitle BirthYear  \
0                              [소설가[小說家], 작가(사람)[作家]]      1931   
1   [연구원(연구자)[硏究員], 문학 평론가[文學評論家], 작가(사람)[作家], 수필가...      1936   
2                                          교수(대학)[敎授]      1965   
3                                          교수(대학)[敎授]      1969   
4          

In [112]:
# top_authors_df.to_excel('./top_authors_df.xlsx', index=False)

In [34]:
len(unmatched_authors)

792

In [35]:
# 매칭 안된건 확인
# ref_df[ref_df['matched_ids'].apply(lambda x: all(item is None for item in x))].to_excel('./unmatched_ref_df.xlsx', index=False)
ref_df.dropna(subset=['author'])[ref_df['matched_ids'].apply(lambda x: all(item is None for item in x))]

/tmp/ipykernel_639691/3502407203.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ref_df.dropna(subset=['author'])[ref_df['matched_ids'].apply(lambda x: all(item is None for item in x))]


,refebibl-id,type-code,type-name,title,author,journal-name,pubi-year,volume,isseue,serno,...,site-name,registration-day,conference-name,modern_liter,unified_title,title_index,author_list,matched_results,matched_authors,matched_ids
24,REF061195563,3,단행본,포유강 사람속,구병천,NaN,1992,NaN,NaN,NaN,...,NaN,NaN,NaN,추가,포유강 사람속,114,[구병천],"[(구병천, None)]",[구병천],[None]
80,REF055500862,3,단행본,노년기의 의미와 즐거움,Erikson,NaN,2000,NaN,NaN,NaN,...,NaN,NaN,NaN,추가,노년기의 의미와 즐거움,170,[Erikson],"[(Erikson, None)]",[Erikson],[None]
107,REF062712494,3,단행본,광기의 역사,미셀 푸코,NaN,1991,NaN,NaN,NaN,...,NaN,NaN,NaN,추가,성의 역사,196,[미셀 푸코],"[(미셀 푸코, None)]",[미셀 푸코],[None]
113,REF062712500,3,단행본,백낙청 회화록4,백낙청 회화록 간행위원회,NaN,2007,NaN,NaN,NaN,...,NaN,NaN,NaN,추가,백낙청 회화록4,202,[백낙청 회화록 간행위원회],"[(백낙청 회화록 간행위원회, None)]",[백낙청 회화록 간행위원회],[None]
118,REF062712505,3,단행본,예술사 구술 총서5-박완서,수류산방 편집부,NaN,2012,NaN,NaN,NaN,...,NaN,NaN,NaN,추가,예술사 구술 총서5-박완서,207,[수류산방 편집부],"[(수류산방 편집부, None)]",[수류산방 편집부],[None]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11007,REF064216980,3,단행본,돌봄선언,더 케어 컬렉티브,NaN,2021,NaN,NaN,NaN,...,NaN,NaN,NaN,기존,돌봄선언,1763,[더 케어 컬렉티브],"[(더 케어 컬렉티브, None)]",[더 케어 컬렉티브],[None]
11041,REF064216621,3,단행본,분단의 역사인식과 사유를 넘어,역사문제연구소 한독비교사포럼,NaN,2019,NaN,NaN,NaN,...,NaN,NaN,NaN,기존,분단의 역사인식과 사유를 넘어,8756,[역사문제연구소 한독비교사포럼],"[(역사문제연구소 한독비교사포럼, None)]",[역사문제연구소 한독비교사포럼],[None]
11045,REF064216625,3,단행본,구술사로 읽는 한국전쟁,한국구술사학회,NaN,2011,NaN,NaN,NaN,...,NaN,NaN,NaN,기존,구술사로 읽는 한국전쟁,4238,[한국구술사학회],"[(한국구술사학회, None)]",[한국구술사학회],[None]
11051,REF064216631,3,단행본,"Haunting the Korean diaspora : shame, secrecy,...","Cho, Grace M",NaN,2008,NaN,NaN,NaN,...,NaN,NaN,NaN,기존,"Haunting the Korean diaspora : shame, secrecy,...",8763,"[Cho, Grace M]","[(Cho, Grace M, None)]","[Cho, Grace M]",[None]


In [51]:
unmatched_ref_df_labeled = pd.read_excel('./unmatched_ref_df_labeled.xlsx')
unmatched_ref_df_labeled

,refebibl-id,type-code,type-name,title,author,journal-name,pubi-year,volume,isseue,serno,...,site-name,registration-day,conference-name,modern_liter,unified_title,title_index,author_list,matched_results,matched_authors,matched_ids
0,REF064216002,3,단행본,남자다움에 관하여,하비 맨스필드,NaN,2010.0,NaN,NaN,NaN,...,NaN,NaN,NaN,추가,남자다움에 관하여,486,['하비 맨스필드'],"[('하비 맨스필드', None)]",['하비 맨스필드'],"[""nlk:KAC200803231""]"
1,REF064216002,3,단행본,남자다움에 관하여,하비 맨스필드,NaN,2010.0,NaN,NaN,NaN,...,NaN,NaN,NaN,기존,남자다움에 관하여,486,['하비 맨스필드'],"[('하비 맨스필드', None)]",['하비 맨스필드'],"[""nlk:KAC200803231""]"
2,REF012857212,1,학술지(정기간행물),≪무소의 뿔처럼 혼자서 가라≫,______,NaN,1993.0,NaN,NaN,NaN,...,NaN,NaN,NaN,기존,≪무소의 뿔처럼 혼자서 가라≫,6133,['______'],"[('______', None)]",['______'],"[""nlk:KAC201007577""]"
3,REF012903474,1,학술지(정기간행물),<꿈꾸는 인큐베이터> ≪한말씀만 하소서≫,______,NaN,1994.0,NaN,NaN,NaN,...,NaN,NaN,NaN,기존,<꿈꾸는 인큐베이터> ≪한말씀만 하소서≫,6135,['______'],"[('______', None)]",['______'],"[""nlk:KAC201007995""]"
4,REF064216893,3,단행본,"1.20 학병사기, 1권",1.20 동지회,NaN,1987.0,NaN,NaN,NaN,...,NaN,NaN,NaN,기존,"1.20 학병사기, 1권",8038,['1.20 동지회'],"[('1.20 동지회', None)]",['1.20 동지회'],[None]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
965,REF058438287,1,학술지(정기간행물),Affective encounters: tools of interruption fo...,"Zarzycka, Marta",Feminist Media Studies,2017.0,NaN,NaN,NaN,...,NaN,NaN,NaN,기존,Affective encounters: tools of interruption fo...,7144,"['Zarzycka, Marta']","[('Zarzycka, Marta', None)]","['Zarzycka, Marta']",[None]
966,REF047603318,3,단행본,여성․문화․사회,"Zimbalist, M",NaN,2009.0,NaN,NaN,NaN,...,NaN,NaN,NaN,기존,여성․문화․사회,2618,"['Zimbalist, M']","[('Zimbalist, M', None)]","['Zimbalist, M']","[""nlk:KAC199623423"", ""nlk:KAC200900722""]"
967,REF037923423,3,단행본,불가능한 것의 가능성,"Zizek, S",NaN,2012.0,NaN,NaN,NaN,...,NaN,NaN,NaN,기존,불가능한 것의 가능성,4047,"['Zizek, S']","[('Zizek, S', None)]","['Zizek, S']","[""nlk:KAC199630983""]"
968,REF037929343,3,단행본,Looking awry,"Žižek, Salvoj",NaN,1991.0,NaN,NaN,NaN,...,NaN,NaN,NaN,기존,Looking awry,3840,"['Žižek, Salvoj']","[('Žižek, Salvoj', None)]","['Žižek, Salvoj']","[""nlk:KAC199630983""]"


In [52]:
unmatched_ref_df_labeled = unmatched_ref_df_labeled.drop_duplicates('refebibl-id')

In [53]:
# 휴먼 라벨링 matched_ids 결과를 토대로 author_list와 matched_results 업데이트
# nan 값을 먼저 None으로 변환
unmatched_ref_df_labeled['matched_ids'] = unmatched_ref_df_labeled['matched_ids'].apply(lambda x: 'None' if pd.isna(x) else x)

# ast.literal_eval 적용
unmatched_ref_df_labeled['matched_ids'] = unmatched_ref_df_labeled['matched_ids'].progress_apply(ast.literal_eval)

/tmp/ipykernel_639691/942485316.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unmatched_ref_df_labeled['matched_ids'] = unmatched_ref_df_labeled['matched_ids'].apply(lambda x: 'None' if pd.isna(x) else x)


저자 매칭 중:   0%|          | 0/936 [00:00<?, ?it/s]

/tmp/ipykernel_639691/942485316.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unmatched_ref_df_labeled['matched_ids'] = unmatched_ref_df_labeled['matched_ids'].progress_apply(ast.literal_eval)


In [54]:
# @id와 prefLabel로 구성된 매핑 딕셔너리 생성
id_to_author = dict(zip(df['@id'], df['prefLabel']))

# matched_ids 기반으로 matched_authors 업데이트 함수
def update_matched_authors(matched_ids):
    matched_authors = []
    if isinstance(matched_ids, list):
        for matched_id in matched_ids:
            if matched_id in id_to_author:
                matched_authors.append(id_to_author[matched_id])
    return matched_authors

# 진행 상황을 보여주며 unmatched_ref_df_labeled의 각 matched_ids 리스트에 대해 매칭 수행
tqdm.pandas(desc="저자 매칭 중")
unmatched_ref_df_labeled['matched_authors'] = unmatched_ref_df_labeled['matched_ids'].progress_apply(update_matched_authors)

저자 매칭 중:   0%|          | 0/936 [00:00<?, ?it/s]

/tmp/ipykernel_639691/3348938525.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unmatched_ref_df_labeled['matched_authors'] = unmatched_ref_df_labeled['matched_ids'].progress_apply(update_matched_authors)


In [55]:
unmatched_ref_df_labeled[['refebibl-id','matched_results', 'matched_authors','matched_ids']]

,refebibl-id,matched_results,matched_authors,matched_ids
0,REF064216002,"[('하비 맨스필드', None)]","[Mansfield, Harvey C.]",[nlk:KAC200803231]
2,REF012857212,"[('______', None)]",[공지영],[nlk:KAC201007577]
3,REF012903474,"[('______', None)]",[박완서],[nlk:KAC201007995]
4,REF064216893,"[('1.20 동지회', None)]",[],[None]
5,REF031143328,"[('5․18 기념재단', None)]",[],[nlk:KAB201800311]
...,...,...,...,...
965,REF058438287,"[('Zarzycka, Marta', None)]",[],[None]
966,REF047603318,"[('Zimbalist, M', None)]","[Rosaldo, Michelle Zimbalist, Lamphere, Louise]","[nlk:KAC199623423, nlk:KAC200900722]"
967,REF037923423,"[('Zizek, S', None)]","[Žižek, Slavoj]",[nlk:KAC199630983]
968,REF037929343,"[('Žižek, Salvoj', None)]","[Žižek, Slavoj]",[nlk:KAC199630983]


In [57]:
# ref_df와 unmatched_ref_df_labeled 병합
ref_df_updated = pd.merge(ref_df, unmatched_ref_df_labeled[['refebibl-id', 'matched_authors', 'matched_ids']], on='refebibl-id', how='left', suffixes=('', '_new'))

# 병합 후 새로운 컬럼으로 업데이트
ref_df_updated['matched_authors'] = ref_df_updated['matched_authors_new'].combine_first(ref_df_updated['matched_authors'])
ref_df_updated['matched_ids'] = ref_df_updated['matched_ids_new'].combine_first(ref_df_updated['matched_ids'])

# 불필요한 컬럼 제거
ref_df_updated.drop(columns=['matched_authors_new', 'matched_ids_new'], inplace=True)

In [56]:
len(ref_df)

11092

In [61]:
len(ref_df_updated)

11092

In [60]:
ref_df_updated[ref_df_updated.duplicated('refebibl-id', keep=False)]

,refebibl-id,type-code,type-name,title,author,journal-name,pubi-year,volume,isseue,serno,...,site-name,registration-day,conference-name,modern_liter,unified_title,title_index,author_list,matched_results,matched_authors,matched_ids


### ref_df_updated

In [2]:
ref_df_updated = pd.read_pickle('./240731_ref_df_updated.pkl')

In [3]:
# 'nlk:KAB'로 시작하는 matched_ids 필터링 및 매칭
def match_pref_label(row):
    matched_authors = row['matched_authors'] if 'matched_authors' in row and isinstance(row['matched_authors'], list) else []
    
    if not matched_authors:
        matched_ids = row['matched_ids']
        if matched_ids:
            for matched_id in matched_ids:
                if matched_id and isinstance(matched_id, str) and matched_id.startswith('nlk:KAB'):
                    matching_row = df_org[df_org['@id'] == matched_id]
                    if not matching_row.empty:
                        corporate_name = matching_row['corporateName'].values[0] if pd.notna(matching_row['corporateName'].values[0]) else matching_row['prefLabel'].values[0]
                        matched_authors.append(corporate_name)
    
    return matched_authors

# matched_authors 컬럼 업데이트
tqdm.pandas(desc="저자 매칭 중")
ref_df_updated['matched_authors'] = ref_df_updated.progress_apply(match_pref_label, axis=1)

저자 매칭 중:   0%|          | 0/11092 [00:00<?, ?it/s]

In [4]:
# '이상경'이 포함된 행에서 matched_ids를 ['KAC201320882']로 업데이트
ref_df_updated.loc[ref_df_updated['author'].str.contains('이상경', na=False), 'matched_ids'] = ref_df_updated.loc[ref_df_updated['author'].str.contains('이상경', na=False), 'matched_ids'].apply(lambda x: ['nlk:KAC201320882'])

# '이상경'이 포함된 행에서 matched_authors를 ['이상경']으로 업데이트
ref_df_updated.loc[ref_df_updated['author'].str.contains('이상경', na=False), 'matched_authors'] = ref_df_updated.loc[ref_df_updated['author'].str.contains('이상경', na=False), 'matched_authors'].apply(lambda x: ['이상경'])

In [129]:
# '신샛별'이 포함된 행에서 matched_ids를 ['KAC201320882']로 업데이트
ref_df_updated.loc[ref_df_updated['author'].str.contains('신샛별', na=False), 'matched_ids'] = ref_df_updated.loc[ref_df_updated['author'].str.contains('신샛별', na=False), 'matched_ids'].apply(lambda x: ['nlk:KAC신샛별'])

# '신샛별'이 포함된 행에서 matched_authors를 ['신샛별']으로 업데이트
ref_df_updated.loc[ref_df_updated['author'].str.contains('신샛별', na=False), 'matched_authors'] = ref_df_updated.loc[ref_df_updated['author'].str.contains('신샛별', na=False), 'matched_authors'].apply(lambda x: ['신샛별'])

In [6]:
df_year = pd.read_pickle('./240731_2008_2022_한국현대문학_통합_성별_생년.pkl')

In [7]:
df_year

,article-id,pub-year,gender,birth
0,ART002536712,2019,남,69
1,ART002536697,2019,남,71
2,ART002536682,2019,여,78
3,ART002536671,2019,남,74
4,ART002536636,2019,남,90
...,...,...,...,...
15922,ART002796203,2021,NaN,NaN
15923,ART001536392,2010,여,70
15924,ART001536458,2010,여,65
15925,ART001941069,2014,남,50


In [8]:
ref_df_updated = ref_df_updated.merge(df_year[['article-id','pub-year']], left_on='artiId', right_on='article-id')

In [ ]:
ref_df_updated['pub-year'] = ref_df_updated['pub-year'].astype(int)

In [131]:
# ref_df_updated.to_pickle('./240731_ref_df_updated.pkl')
# ref_df_updated.to_excel('./240731_ref_df_updated.xlsx',index=False)